In [33]:
import numpy as np
import random
from collections import Counter


def create_feature_sets_and_labels(test_size = 0.2):

    # 5 features
    features = []
    features.append([[0, 0, 0, 0, 0], [0]])
    features.append([[0, 0, 0, 0, 1], [0]])
    features.append([[0, 0, 0, 1, 1], [0]])
    features.append([[0, 0, 1, 1, 1], [0]])
    features.append([[0, 1, 1, 1, 1], [0]])
    features.append([[1, 1, 1, 1, 0], [0]])
    features.append([[1, 1, 1, 0, 0], [0]])
    features.append([[1, 1, 0, 0, 0], [0]])
    features.append([[1, 0, 0, 0, 0], [0]])
    features.append([[1, 0, 0, 1, 0], [0]])
    features.append([[1, 0, 1, 1, 0], [0]])
    features.append([[1, 1, 0, 1, 0], [0]])
    features.append([[0, 1, 0, 1, 1], [0]])
    features.append([[0, 0, 1, 0, 1], [0]])
    # output of [1] of positions [0,4]==1
    features.append([[1, 0, 0, 0, 1], [1]])
    features.append([[1, 1, 0, 0, 1], [1]])
    features.append([[1, 1, 1, 0, 1], [1]])
    features.append([[1, 1, 1, 1, 1], [1]])
    features.append([[1, 0, 0, 1, 1], [1]])
    features.append([[1, 0, 1, 1, 1], [1]])
    features.append([[1, 1, 0, 1, 1], [1]])
    features.append([[1, 0, 1, 0, 1], [1]])

    random.shuffle(features)
    features = np.array(features)

    testing_size = int(test_size*len(features))

    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    return train_x,train_y,test_x,test_y

if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_feature_sets_and_labels()

train_x, train_y

([[0, 0, 1, 0, 1],
  [0, 0, 0, 0, 0],
  [1, 1, 0, 1, 1],
  [0, 1, 0, 1, 1],
  [1, 1, 0, 1, 0],
  [0, 1, 1, 1, 1],
  [1, 0, 1, 1, 0],
  [0, 0, 0, 0, 1],
  [1, 0, 1, 1, 1],
  [1, 1, 0, 0, 1],
  [0, 0, 0, 1, 1],
  [1, 0, 0, 1, 1],
  [1, 0, 1, 0, 1],
  [0, 0, 1, 1, 1],
  [1, 1, 1, 1, 0],
  [1, 1, 0, 0, 0],
  [1, 0, 0, 0, 0],
  [1, 1, 1, 1, 1]],
 [[0],
  [0],
  [1],
  [0],
  [0],
  [0],
  [0],
  [0],
  [1],
  [1],
  [0],
  [1],
  [1],
  [0],
  [0],
  [0],
  [0],
  [1]])

In [57]:
import tensorflow as tf
import numpy as np

train_x,train_y,test_x,test_y = create_feature_sets_and_labels()

n_nodes_hl1 = 20
n_nodes_hl2 = 20

n_classes = 1
hm_epochs = 10

x = tf.placeholder('float')
y = tf.placeholder('float')

hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


def neural_network_model(data):

    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.sigmoid(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.sigmoid(l2)

    output = tf.matmul(l2,output_layer['weight']) + output_layer['bias']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.0001, beta1=0.9, beta2=0.999).minimize(cost)
    optimizer = tf.train.GradientDescentOptimizer(1).minimize(cost)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
  
        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while i < len(train_x):
                start = i
                end = i+batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                epoch_loss += c
                i+=batch_size
                last_cost = c

            if (epoch% 2) == 0 and epoch > 1:
                print('Epoch', epoch+1, 'completed out of',hm_epochs,'cost:', last_cost)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

train_neural_network(x)

Epoch 3 completed out of 10 cost: 0.0
Epoch 5 completed out of 10 cost: 0.0
Epoch 7 completed out of 10 cost: 0.0
Epoch 9 completed out of 10 cost: 0.0
Accuracy: 1.0
